In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gpt-oss:20b", temperature=0)

In [3]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [5]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt
)

In [7]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [8]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='a0661d97-265c-4543-9255-99ae42322c61'),
              AIMessage(content='To answer this question, I will call the search_web function with the query "langchain-mcp-adapters library".\n\n{"name": "search_web", "parameters": {"query": "langchain-mcp-adapters library"}}', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-03T20:17:20.694056346Z', 'done': True, 'done_reason': 'stop', 'total_duration': 67940149424, 'load_duration': 17106627076, 'prompt_eval_count': 297, 'prompt_eval_duration': 36837136701, 'eval_count': 47, 'eval_duration': 13862697937, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b8580-d9cc-7eb1-8ee9-8215c348a7f6-0', usage_metadata={'input_tokens': 297, 'output_tokens': 47, 'total_tokens': 344})]}


## Online MCP

In [9]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [10]:
agent = create_agent(
    model=model,
    tools=tools,
)

In [11]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='f8f09108-256e-43a8-8047-a2b4d247cf81'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-03T20:22:15.514302358Z', 'done': True, 'done_reason': 'stop', 'total_duration': 48337020616, 'load_duration': 205435868, 'prompt_eval_count': 363, 'prompt_eval_duration': 42056111853, 'eval_count': 21, 'eval_duration': 6008899074, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019b8585-a5f4-7522-bb02-213ac884ab66-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'America/New_York'}, 'id': '17b731cc-2154-4f43-aa1a-133e720594ae', 'type': 'tool_call'}], usage_metadata={'input_tokens': 363, 'output_tokens': 21, 'total_tokens': 384}),
              ToolMessage(content=[{'type': 'text', 'text': '{\n  "timezone": "America/New_York",\n  "datetime": "2026-01-03T15:22:16-05:00"